In [ ]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q torch
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q pyarrow==13.0.0
!pip install -q datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:

In [ ]:
from google.colab import userdata

# Defined in the secrets tab in Google Colab
hf_token = userdata.get('HuggingFace')

In [ ]:
from huggingface_hub import whoami

user = whoami(token=hf_token)['name']

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import random

from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

In [ ]:
from typing import TypeVar, List, Tuple, Dict, NoReturn
from datasets import DatasetDict, Dataset, IterableDatasetDict, IterableDataset

T = TypeVar("T")

DataPointType=DatasetDict | Dataset | IterableDatasetDict | IterableDataset
DataPointTrain =Dataset | List | T
DataPointTest =Dataset | List | T

from transformers.models.llama.tokenization_llama_fast import LlamaTokenizerFast
from transformers.models.mistral.modeling_mistral import MistralForCausalLM

In [ ]:
# BASE_MODEL_NAME = "mistralai/Mistral-7B-v0.1"
BASE_MODEL_NAME="mistralai/Mistral-7B-Instruct-v0.2"

ABSTRACT_LAYER_ADAPTER_CHECKPOINTS_NAME = "ckpts_mistral_7B_abstract_layer_planning_decompn"
DETAIL_LAYER_ADAPTER_CHECKPOINTS_NAME = "ckpts_mistral_7B_detail_layer_planning_decompn"

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
from transformers import AutoTokenizer

def load_tokenizer(model_id: str = BASE_MODEL_NAME)->LlamaTokenizerFast:
    tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

    # Check if the pad token is already in the tokenizer
    if '[<pad>]' not in tokenizer.get_vocab():
        # Add the pad token
        tokenizer.add_tokens(['[<pad>]'])
        print("Added pad token")

    # Set the pad token
    tokenizer.pad_token = '[<pad>]'

    tokenizer.padding_side='left'

    # Print the pad token ids
    print('Tokenizer pad token ID:', tokenizer.pad_token_id)
    return tokenizer

In [ ]:
tokenizer=load_tokenizer()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Added pad token
Tokenizer pad token ID: 32000


In [ ]:
print("EOS token:", tokenizer.eos_token)
print("EOS token id:", tokenizer.eos_token_id)

print("Pad token: ", tokenizer.pad_token)
print("Pad token ID: ", tokenizer.pad_token_id)

print(tokenizer)

EOS token: </s>
EOS token id: 2
Pad token:  [<pad>]
Pad token ID:  32000
LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-Instruct-v0.2', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '[<pad>]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("[<pad>]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
}


In [ ]:
from datasets import load_dataset

def custom_dataset_load(data_dir: str)->DataPointType:
    data = load_dataset(data_dir, split="train")
    return data

In [ ]:
def format_dataset_abstract_fn(data_points: DataPointType)->DataPointType:
    system_prompt="Below is an instruction that describes a composite task. Your response should fulfill the instruction by providing a structured breakdown of the composite task into distinct subtasks."
    INST="### Instruction:"
    RESP="### Response:"

    # Create the formatted text
    formatted_dataset = data_points.map(
        lambda x: {
            "prompt": "".join(
                [
                    f"{system_prompt}\n\n",
                    f"{INST}{x['instruction'].strip()}\n\n",
                    f"{RESP}{x['output-abstract'].strip()}",
                ]
            ),
            "response": "".join(
                [
                    f"{RESP}{x['output-abstract'].strip()}",
                ]
            ),
        }
    )
    return formatted_dataset

In [ ]:
def format_dataset_abstract_instruct_fn(data_points: DataPointType)->DataPointType:
    template="Below is an instruction that describes a composite task. Your response should fulfill the instruction by providing a structured breakdown of the composite task into distinct subtasks."
    B_INST,E_INST="[INST]","[/INST]"
    RESP="### Response:"

    # Create the formatted text
    formatted_dataset = data_points.map(
        lambda x: {
            "prompt": "".join(
                [
                    f"{B_INST}{template.strip()}{x['instruction'].strip()}{E_INST}\n\n",
                    f"{RESP}{x['output-abstract'].strip()}",
                ]
            ),
            "response": "".join(
                [
                    f"{RESP}{x['output-abstract'].strip()}",
                ]
            ),
        }
    )
    return formatted_dataset

In [ ]:
def format_dataset_detail_fn(data_points: DataPointType)->DataPointType:
    system_prompt="Below is an instruction that describes a composite task. Your response should fulfill the instruction by providing a structured breakdown of the composite task into distinct subtasks."
    INST="### Instruction:"
    RESP="### Response:"

    # Create the formatted text
    formatted_dataset = data_points.map(
        lambda x: {
            "prompt": "".join(
                [
                    f"{system_prompt}\n\n",
                    f"{INST}{x['instruction'].strip()}\n\n",
                    f"{RESP}{x['output-detail'].strip()}",
                ]
            ),
            "response": "".join(
                [
                    f"{RESP}{x['output-detail'].strip()}",
                ]
            ),
        }
    )
    return formatted_dataset

In [ ]:
def format_dataset_detail_instruct_fn(data_points: DataPointType)->DataPointType:
    template="Below is an instruction that describes a composite task. Your response should fulfill the instruction by providing a structured breakdown of the composite task into distinct subtasks."
    B_INST,E_INST="[INST]","[/INST]"
    RESP="### Response:"

    # Create the formatted text
    formatted_dataset = data_points.map(
        lambda x: {
            "prompt": "".join(
                [
                    f"{B_INST}{template.strip()}{x['instruction'].strip()}{E_INST}\n\n",
                    f"{RESP}{x['output-detail'].strip()}",
                ]
            ),
            "response": "".join(
                [
                    f"{RESP}{x['output-detail'].strip()}",
                ]
            ),
        }
    )
    return formatted_dataset

In [ ]:
def shuffle_tokenize_batch(data:DataPointType, tokenizer:LlamaTokenizerFast)->DataPointType:
    data = data.shuffle(seed=1234)  # Shuffle dataset here
    data = data.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
    return data

In [ ]:
def split_train_test_dataset(data:DataPointType, split_size: float = 0.1)->Tuple[DataPointTrain,DataPointTest]:
    data = data.train_test_split(test_size=split_size)
    train_data = data["train"]
    test_data = data["test"]
    return train_data, test_data

In [ ]:
def format_prediction_fn(data_point: DataPointType)->str:
    system_prompt="Below is an instruction that describes a composite task. Your response should fulfill the instruction by providing a structured breakdown of the composite task into distinct subtasks."
    INST="### Instruction:"
    RESP="### Response:"

    # Create the formatted text
    formatted_prediction = "".join(
                [
                    f"{system_prompt}\n\n",
                    f"{INST}{data_point['instruction'].strip()}\n\n",
                    f"{RESP}",
                ]
            )
    return formatted_prediction

In [ ]:
def format_prediction_instruct_fn(data_point: DataPointType)->str:
    template="Below is an instruction that describes a composite task. Your response should fulfill the instruction by providing a structured breakdown of the composite task into distinct subtasks."
    B_INST,E_INST="[INST]","[/INST]"
    RESP="### Response:"

    # Create the formatted text
    formatted_prediction = "".join(
                [
                    f"{B_INST}{template.strip()}{data_point['instruction'].strip()}{E_INST}\n\n",
                    f"{RESP}",
                ]
            )
    return formatted_prediction

In [ ]:
def format_prediction_abstr_instruct_fn(data_point: DataPointType)->str:
    # template="Below is an instruction that describes a composite task. Your response should fulfill the instruction by providing a structured breakdown of the composite task into abstract distinct subtasks using only keywords."
    # template="Decompose the following instruction to actionable subtasks. Provide ONLY the keywords for each subtask with NO additional information."
    template="Decompose the following instruction to actionable subtasks. Provide ONLY ONE keyword for each subtask with NO additional information."

    B_INST,E_INST="[INST]","[/INST]"
    RESP="### Response:"

    # Create the formatted text
    formatted_prediction = "".join(
                [
                    f"{B_INST}{template.strip()}{data_point['instruction'].strip()}{E_INST}\n\n",
                    f"{RESP}",
                ]
            )
    return formatted_prediction

In [ ]:
def format_prediction_detail_instruct_fn(data_point: DataPointType)->str:
    # template="Below is an instruction that describes a composite task. Your response should fulfill the instruction by providing a structured breakdown of the composite task into detailed distinct subtasks."
    template="Decompose the following instruction to actionable subtasks. Provide detailed explanations for each decomposed subtask."
    B_INST,E_INST="[INST]","[/INST]"
    RESP="### Response:"

    # Create the formatted text
    formatted_prediction = "".join(
                [
                    f"{B_INST}{template.strip()}{data_point['instruction'].strip()}{E_INST}\n\n",
                    f"{RESP}",
                ]
            )
    return formatted_prediction

In [ ]:
import torch
from transformers import StoppingCriteria

class KeywordsStoppingCriteria(StoppingCriteria):
    def __init__(self, keywords, tokenizer, input_ids):
        self.keywords = keywords
        self.keyword_ids = []
        self.max_keyword_len = 0
        for keyword in keywords:
            cur_keyword_ids = tokenizer(keyword).input_ids
            if len(cur_keyword_ids) > 1 and cur_keyword_ids[0] == tokenizer.bos_token_id:
                cur_keyword_ids = cur_keyword_ids[1:]
            if len(cur_keyword_ids) > self.max_keyword_len:
                self.max_keyword_len = len(cur_keyword_ids)
            self.keyword_ids.append(torch.tensor(cur_keyword_ids))
        self.tokenizer = tokenizer
        self.start_len = input_ids.shape[1]

    def call_for_batch(self, output_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        offset = min(output_ids.shape[1] - self.start_len, self.max_keyword_len)
        self.keyword_ids = [keyword_id.to(output_ids.device) for keyword_id in self.keyword_ids]
        for keyword_id in self.keyword_ids:
            if (output_ids[0, -keyword_id.shape[0]:] == keyword_id).all():
                return True
        outputs = self.tokenizer.batch_decode(output_ids[:, -offset:], skip_special_tokens=True)[0]
        for keyword in self.keywords:
            if keyword in outputs:
                return True
        return False

    def __call__(self, output_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        outputs = []
        for i in range(output_ids.shape[0]):
            outputs.append(self.call_for_batch(output_ids[i].unsqueeze(0), scores))
        return all(outputs)

In [ ]:
def get_completion(prompt:str, model:MistralForCausalLM, tokenizer:LlamaTokenizerFast, keywords:List[str], keyword_stopping_criteria:KeywordsStoppingCriteria)->str:
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)

    stopping_criteria = keyword_stopping_criteria(keywords, tokenizer, input_ids)

    with torch.inference_mode():
      output_ids = model.generate(
          inputs=input_ids,
          use_cache=True,
          max_new_tokens=512,
          stopping_criteria=[stopping_criteria],
          attention_mask=None,
          do_sample=False,
          output_attentions=True,
          output_scores=True,
          return_dict_in_generate=True,
      )

    output = tokenizer.decode(output_ids['sequences'][0, input_ids.shape[1]:],skip_spectial_tokens=True).strip()

    return output

In [ ]:
data=custom_dataset_load('/content/drive/MyDrive/data_multi_layer_planning')

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
data_abstr=format_dataset_abstract_instruct_fn(data)

Map:   0%|          | 0/1017 [00:00<?, ? examples/s]

In [ ]:
data_detail=format_dataset_detail_instruct_fn(data)

Map:   0%|          | 0/1017 [00:00<?, ? examples/s]

In [ ]:
processed_data_abstr=shuffle_tokenize_batch(data_abstr,tokenizer)

Map:   0%|          | 0/1017 [00:00<?, ? examples/s]

In [ ]:
processed_data_detail=shuffle_tokenize_batch(data_detail,tokenizer)

Map:   0%|          | 0/1017 [00:00<?, ? examples/s]

In [ ]:
processed_df_abstr = processed_data_abstr.to_pandas()
processed_df_abstr.head(5)

,instruction,output-detail,output-abstract,prompt,response,input_ids,attention_mask
0,Produce a series of educational videos on sust...,1. DIY Recycling Projects: Create step-by-step...,"1. DIY Recycling Projects, 2. Downloadable Gui...",[INST]Below is an instruction that describes a...,"### Response:1. DIY Recycling Projects, 2. Dow...","[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,Design a home renovation guide including 3D mo...,1. 3D Models: Create detailed 3D renderings of...,"1. 3D Models, 2. DIY Tutorial Videos, 3. Writt...",[INST]Below is an instruction that describes a...,"### Response:1. 3D Models, 2. DIY Tutorial Vid...","[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"Organize a virtual conference, including a web...",1. Conference Website: Design a website listin...,"1. Conference website, 2. Promotional video, 3...",[INST]Below is an instruction that describes a...,"### Response:1. Conference website, 2. Promoti...","[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,Produce a digital art exhibition exploring the...,1. Digital Paintings: A series of high-resolut...,"1. Digital Paintings, 2. Explanatory Guide, 3....",[INST]Below is an instruction that describes a...,"### Response:1. Digital Paintings, 2. Explanat...","[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,Launch a fashion line including the collection...,1. Collection Theme: Develop a captivating nar...,"1. Collection Theme, 2. Fashion Sketches, 3. R...",[INST]Below is an instruction that describes a...,"### Response:1. Collection Theme, 2. Fashion S...","[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
processed_df_detail = processed_data_detail.to_pandas()
processed_df_detail.head(5)

,instruction,output-detail,output-abstract,prompt,response,input_ids,attention_mask
0,Produce a series of educational videos on sust...,1. DIY Recycling Projects: Create step-by-step...,"1. DIY Recycling Projects, 2. Downloadable Gui...",[INST]Below is an instruction that describes a...,### Response:1. DIY Recycling Projects: Create...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,Design a home renovation guide including 3D mo...,1. 3D Models: Create detailed 3D renderings of...,"1. 3D Models, 2. DIY Tutorial Videos, 3. Writt...",[INST]Below is an instruction that describes a...,### Response:1. 3D Models: Create detailed 3D ...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"Organize a virtual conference, including a web...",1. Conference Website: Design a website listin...,"1. Conference website, 2. Promotional video, 3...",[INST]Below is an instruction that describes a...,### Response:1. Conference Website: Design a w...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,Produce a digital art exhibition exploring the...,1. Digital Paintings: A series of high-resolut...,"1. Digital Paintings, 2. Explanatory Guide, 3....",[INST]Below is an instruction that describes a...,### Response:1. Digital Paintings: A series of...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,Launch a fashion line including the collection...,1. Collection Theme: Develop a captivating nar...,"1. Collection Theme, 2. Fashion Sketches, 3. R...",[INST]Below is an instruction that describes a...,### Response:1. Collection Theme: Develop a ca...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
_,test_data_abstr=split_train_test_dataset(processed_data_abstr)

In [ ]:
_,test_data_detail=split_train_test_dataset(processed_data_detail)

In [ ]:
import torch
from transformers import AutoModelForCausalLM,BitsAndBytesConfig

def load_model(model_id: str = BASE_MODEL_NAME,device_map:Dict[str,int]={"": 0})->MistralForCausalLM:

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_id, quantization_config=bnb_config, device_map=device_map
    )
    return model

In [ ]:
model=load_model()

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
# Resize token embeddings
model.resize_token_embeddings(len(tokenizer))

# Update pad token id in model and its config
model.pad_token_id = tokenizer.pad_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# Check if they are equal
assert model.pad_token_id == tokenizer.pad_token_id, "The model's pad token ID does not match the tokenizer's pad token ID"

In [ ]:
model.pad_token_id=tokenizer.pad_token_id
model.config.pad_token_id=tokenizer.pad_token_id

model.config.use_cache = True

In [ ]:
stop_keywords = ["</s>"," </s>", "</s> "]

In [ ]:
pred_prompt=format_prediction_instruct_fn(test_data_abstr[0])
print(pred_prompt)

[INST]Below is an instruction that describes a composite task. Your response should fulfill the instruction by providing a structured breakdown of the composite task into distinct subtasks.Develop a virtual museum exhibit on Ancient Egypt, featuring a 3D tour (video), educational content on artifacts (text), narrated historical timelines (audio), and interactive quizzes (text).[/INST]

### Response:


In [ ]:
pred_prompt_abstr=format_prediction_abstr_instruct_fn(test_data_abstr[0])
print(pred_prompt_abstr)

[INST]Decompose the following instruction to actionable subtasks. Provide ONLY ONE keyword for each subtask with NO additional information.Develop a virtual museum exhibit on Ancient Egypt, featuring a 3D tour (video), educational content on artifacts (text), narrated historical timelines (audio), and interactive quizzes (text).[/INST]

### Response:


In [ ]:
pred_prompt_detail=format_prediction_detail_instruct_fn(test_data_abstr[0])
print(pred_prompt_detail)

[INST]Decompose the following instruction to actionable subtasks. Provide detailed explanations for each decomposed subtask.Develop a virtual museum exhibit on Ancient Egypt, featuring a 3D tour (video), educational content on artifacts (text), narrated historical timelines (audio), and interactive quizzes (text).[/INST]

### Response:


In [ ]:
output_base=get_completion(pred_prompt, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)
print(output_base)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
MistralModel is using MistralSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


[<pad>]: Virtual Museum Exhibit on Ancient Egypt

To develop a virtual museum exhibit on Ancient Egypt featuring a 3D tour, educational content on artifacts, narrated historical timelines, and interactive quizzes, follow this structured breakdown of distinct subtasks:

1. **Research and Scripting**
   - Gather comprehensive information on Ancient Egyptian history, culture, and artifacts.
   - Write detailed scripts for the 3D tour, educational content, narrated historical timelines, and interactive quizzes.

2. **3D Modeling and Texturing**
   - Create 3D models of significant Ancient Egyptian artifacts, structures, and landscapes.
   - Texture and light the 3D models to create an authentic[<pad>][<pad>][<pad>] and visually appealing appearance.

3. **Video Production**
   - Film or source a high-quality 3D tour video of the virtual exhibit.
   - Edit the video to include the 3D models, educational content, and narrated historical timelines.

4. **Audio Recording and Editing**
   - Rec

In [ ]:
# with model.disable_adapter(): Use this if you intend to disable adapters (i.e., after merging the adapters with the base model)
output_base_abstr=get_completion(pred_prompt_abstr, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)
print(output_base_abstr)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


1. Research Ancient Egypt (text)
2. Create 3D tour (video production)
3. Write educational content (text)
4. Record narrated historical timelines (audio)
5. Design interactive quizzes (text creation)</s>


In [ ]:
# with model.disable_adapter():
output_base_detail=get_completion(pred_prompt_detail, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)
print(output_base_detail)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


[<pad>]: I. Planning and Research
1. Identify the target audience and their level of knowledge about Ancient Egypt.
2. Conduct thorough research on Ancient Egyptian history, art, culture, and artifacts.
3. Gather multimedia resources, such as images, videos, and audio files, for the exhibit.
4. Create a rough outline of the exhibit's structure, including the 3D tour, textual content, narrated historical timelines, and interactive quizzes.

### II. Creating the 3D Tour (Video)
1. Choose a 3D modeling software or platform to create the virtual museum exhibit.
2. Design and create 3D models of the museum layout, including rooms, exhibits, and artifacts.
3. Texture and light the 3D models to create a visually appealing and immersive environment.
4. Import the 3D models into a video editing software to create the virtual tour.
5. Add transitions, effects, and music to enhance the user experience.

### III. Developing Educational Content on Artifacts (Text)
1. Write informative and engaging 

In [ ]:
adapter_id_abstr='/content/drive/MyDrive/ckpts_mistral_7B_abstract_layer_planning_decompn'
adapter_id_detail="/content/drive/MyDrive/ckpts_mistral_7B_detail_layer_planning_decompn"

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!unzip /content/drive/MyDrive/ckpts_mistral_7B_abstract_layer_planning_decompn.zip -d ckpts_mistral_7B_abstract_layer_planning_decompn

Archive:  /content/drive/MyDrive/ckpts_mistral_7B_abstract_layer_planning_decompn.zip
   creating: ckpts_mistral_7B_abstract_layer_planning_decompn/ckpts_mistral_7B_abstract_layer_planning_decompn/
 extracting: ckpts_mistral_7B_abstract_layer_planning_decompn/ckpts_mistral_7B_abstract_layer_planning_decompn/added_tokens.json  
  inflating: ckpts_mistral_7B_abstract_layer_planning_decompn/ckpts_mistral_7B_abstract_layer_planning_decompn/adapter_config.json  
  inflating: ckpts_mistral_7B_abstract_layer_planning_decompn/ckpts_mistral_7B_abstract_layer_planning_decompn/README.md  
  inflating: ckpts_mistral_7B_abstract_layer_planning_decompn/ckpts_mistral_7B_abstract_layer_planning_decompn/tokenizer.model  
  inflating: ckpts_mistral_7B_abstract_layer_planning_decompn/ckpts_mistral_7B_abstract_layer_planning_decompn/tokenizer_config.json  
  inflating: ckpts_mistral_7B_abstract_layer_planning_decompn/ckpts_mistral_7B_abstract_layer_planning_decompn/special_tokens_map.json  
  inflating: c

In [ ]:
!unzip /content/drive/MyDrive/ckpts_mistral_7B_detail_layer_planning_decompn.zip -d ckpts_mistral_7B_detail_layer_planning_decompn

Archive:  /content/drive/MyDrive/ckpts_mistral_7B_detail_layer_planning_decompn.zip
   creating: ckpts_mistral_7B_detail_layer_planning_decompn/ckpts_mistral_7B_detail_layer_planning_decompn/
 extracting: ckpts_mistral_7B_detail_layer_planning_decompn/ckpts_mistral_7B_detail_layer_planning_decompn/added_tokens.json  
  inflating: ckpts_mistral_7B_detail_layer_planning_decompn/ckpts_mistral_7B_detail_layer_planning_decompn/adapter_config.json  
  inflating: ckpts_mistral_7B_detail_layer_planning_decompn/ckpts_mistral_7B_detail_layer_planning_decompn/README.md  
  inflating: ckpts_mistral_7B_detail_layer_planning_decompn/ckpts_mistral_7B_detail_layer_planning_decompn/tokenizer.model  
  inflating: ckpts_mistral_7B_detail_layer_planning_decompn/ckpts_mistral_7B_detail_layer_planning_decompn/tokenizer_config.json  
  inflating: ckpts_mistral_7B_detail_layer_planning_decompn/ckpts_mistral_7B_detail_layer_planning_decompn/special_tokens_map.json  
  inflating: ckpts_mistral_7B_detail_layer_p

In [ ]:
model = PeftModel.from_pretrained(model,'/content/ckpts_mistral_7B_abstract_layer_planning_decompn/ckpts_mistral_7B_abstract_layer_planning_decompn', adapter_name="abstract")
_ = model.load_adapter('/content/ckpts_mistral_7B_detail_layer_planning_decompn/ckpts_mistral_7B_detail_layer_planning_decompn', adapter_name="detail")

In [ ]:
%%time

adapters = ["abstract", "detail"]
weights = [1.0, 1.0]
adapter_name = "merge"
density = 0.2
combination_type = "ties"
if adapter_name in model.peft_config:
    model.delete_adapter(adapter_name)
model.add_weighted_adapter(adapters, weights, adapter_name, combination_type=combination_type, density=density)

CPU times: user 1.4 s, sys: 69 ms, total: 1.47 s
Wall time: 1.75 s


In [ ]:
model.set_adapter("merge")

In [ ]:
output=get_completion(pred_prompt, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)
print(output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


1. 3D Tour: Create a video tour of the virtual museum exhibit, allowing visitors to explore Ancient Egyptian artifacts in a 3D environment. 2. Educational Content: Write informative text about each artifact, including historical context, cultural significance, and materials used. 3. Narrated Historical Timelines: Record audio narrations of key historical events in Ancient Egyptian history, providing context and background information. 4. Interactive Quizzes: Design text-based quizzes to test visitors' knowledge of Ancient Egyptian history and culture.</s>


In [ ]:
output_abstr=get_completion(pred_prompt_abstr, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)
print(output_abstr)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


1. 3D Tour: Video production.2. Educational Content: Text writing.3. Narrated Timelines: Audio recording.4. Interactive Quizzes: Text design.</s>


In [ ]:
output_detail=get_completion(pred_prompt_detail, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)
print(output_detail)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


1. 3D Tour: Create a 3D model of the museum exhibit, including various artifacts and architectural elements. 2. Educational Content: Write informative texts about each artifact, including historical context, cultural significance, and materials used. 3. Narrated Historical Timelines: Record audio narrations for each timeline, providing a historical context for the artifacts and events. 4. Interactive Quizzes: Design multiple-choice quizzes to test visitors' knowledge of Ancient Egyptian history and culture.</s>


In [ ]:
pred_prompt_abstr_2=format_prediction_abstr_instruct_fn(test_data_abstr[90])
print(pred_prompt_abstr_2)

[INST]Decompose the following instruction to actionable subtasks. Provide ONLY ONE keyword for each subtask with NO additional information.Develop a social media campaign for a new eco-friendly product line, including product photography (image), testimonial videos from early users (video), blog posts highlighting the sustainability aspect (text), and a series of soothing background sounds for IG stories (audio).[/INST]

### Response:


In [ ]:
pred_prompt_detail_2=format_prediction_detail_instruct_fn(test_data_abstr[90])
print(pred_prompt_detail_2)

[INST]Decompose the following instruction to actionable subtasks. Provide detailed explanations for each decomposed subtask.Develop a social media campaign for a new eco-friendly product line, including product photography (image), testimonial videos from early users (video), blog posts highlighting the sustainability aspect (text), and a series of soothing background sounds for IG stories (audio).[/INST]

### Response:


In [ ]:
output_abstr_2=get_completion(pred_prompt_abstr_2, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)
print(output_abstr_2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


1. Product photography: Image.2. Testimonial videos: Video.3. Blog posts: Text.4. Background sounds: Audio.</s>


In [ ]:
output_detail_2=get_completion(pred_prompt_detail_2, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)
print(output_detail_2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


1. Product Photography: Capture high-quality images of the eco-friendly product line. Ensure the photos showcase the unique features and benefits of the products. Use natural lighting and minimalistic backgrounds to emphasize the eco-friendliness.

### 2. Testimonial Videos: Record and edit videos of early users sharing their experiences with the product line. Focus on their positive feedback, the impact the products have had on their lives, and how they align with the eco-friendly values.

### 3. Blog Posts: Write and publish blog posts that highlight the sustainability aspects of the product line. Include information on the materials used, production processes, and the environmental benefits. Use engaging storytelling to connect with readers and encourage sharing.

### 4. Background Sounds: Create a series of soothing background sounds for IG stories. These sounds should be calming, eco-friendly, and complementary to the brand image. Use natural sounds like rain, wind, or birds chirp

In [ ]:
from tqdm import tqdm

outputs_abstract=[]
for i in tqdm(range(len(test_data_abstr))):
  pred_instruct_abstract=format_prediction_abstr_instruct_fn(test_data_abstr[i])

  output_abstract=get_completion(pred_instruct_abstract, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)

  outputs_abstract.append(output_abstract)

print(outputs_abstract)

  0%|          | 0/102 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  1%|          | 1/102 [00:07<12:20,  7.34s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  2%|▏         | 2/102 [00:13<11:21,  6.82s/it]The attention mask and the 

['1. 3D Tour: Video. 2. Educational Content: Text. 3. Narrated Timelines: Audio. 4. Interactive Quizzes: Text.</s>', '1. Introduction video: Video. 2. Daily workout routines: Text/Video. 3. Nutrition guides: Text/Image. 4. Motivational forum: Text.</s>', '1. Logo: Design a unique logo for the fitness influencer.2. Motivational Speech: Record a motivational speech.3. Brand Manifesto: Write a compelling brand manifesto.4. Workout Demonstration: Film a workout demonstration video.</s>', "1. Performances: Live video recordings of artists' sets.\n2. Interviews: Audio recordings of artist interviews.\n3. Q&A sessions: Text-based fan interactions.\n4. Highlights reel: Compilation video of festival moments.</s>", '1. Blog post: Writing, 2. Statistics: Research, 3. Infographics: Design, 4. Podcast episode: Recording.</s>', '1. Written guide, 2. Photo gallery, 3. Music playlists, 4. Walking tour videos.</s>', '1. Scenario-based modules: Video. 2. Instructional guides: Text/image. 3. Communicatio

In [ ]:
outputs_detail=[]
for i in tqdm(range(len(test_data_detail))):
  pred_instruct_detail=format_prediction_detail_instruct_fn(test_data_detail[i])

  output_detail=get_completion(pred_instruct_detail, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)

  outputs_detail.append(output_detail)

print(outputs_detail)

  0%|          | 0/102 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  1%|          | 1/102 [00:30<50:33, 30.04s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  2%|▏         | 2/102 [00:41<31:48, 19.09s/it]The attention mask and the 

['1. 3D Models of Artifacts:\nCreate high-quality 3D models of artifacts from ancient civilizations using computer-aided design (CAD) software. Ensure that the models are accurate and textured to resemble the original artifacts as closely as possible.\n\n### 2. Historical Context and Descriptions:\nResearch and write detailed historical contexts and descriptions for each artifact, including their cultural significance, historical period, and materials used. Ensure that the information is accurate, engaging, and accessible to a wide audience.\n\n### 3. Guided Video Tour:\nCreate a video tour of the virtual museum exhibit, guiding viewers through the exhibit and providing additional information about each artifact. Use a clear and concise narrative style to engage viewers and enhance their understanding of the artifacts and their historical contexts.</s>', '1. Video Tutorials: Record step-by-step video tutorials for each pastry recipe. 2. Written Recipes: Write detailed, easy-to-follow r

In [ ]:
test_data_with_abstr_layer_pred=test_data_abstr.add_column("abstract-layer-pred",outputs_abstract)
test_data_with_detail_layer_pred=test_data_detail.add_column("detail-layer-pred",outputs_detail)

Flattening the indices:   0%|          | 0/102 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/102 [00:00<?, ? examples/s]

In [ ]:
df_test_with_abstr_layer_pred = test_data_with_abstr_layer_pred.to_pandas()
df_test_with_detail_layer_pred = test_data_with_detail_layer_pred.to_pandas()

In [ ]:
df_test_with_abstr_layer_pred.head(5)

,instruction,output-detail,output-abstract,prompt,response,input_ids,attention_mask,abstract-layer-pred
0,Develop a virtual museum exhibit on Ancient Eg...,1. 3D Tour: Design a virtual space replicating...,"1. 3D Tour, 2. Artifacts Content, 3. Narrated ...",[INST]Below is an instruction that describes a...,"### Response:1. 3D Tour, 2. Artifacts Content,...","[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1. 3D Tour: Video. 2. Educational Content: Tex...
1,Assemble a comprehensive fitness challenge onl...,1. Kickoff video: Start with a high-energy vid...,"1. Kickoff video, 2. Daily workouts, 3. Nutrit...",[INST]Below is an instruction that describes a...,"### Response:1. Kickoff video, 2. Daily workou...","[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1. Introduction video: Video. 2. Daily workout...
2,Launch a personal branding package for a fitne...,"1. Logo: Design a dynamic, vibrant logo that r...","1. Logo, 2. Motivational Speech, 3. Brand Mani...",[INST]Below is an instruction that describes a...,"### Response:1. Logo, 2. Motivational Speech, ...","[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1. Logo: Design a unique logo for the fitness ...
3,Organize a virtual music festival featuring va...,1. Live Performances: Schedule and stream live...,"1. Live Performances, 2. Artist Interviews, 3....",[INST]Below is an instruction that describes a...,"### Response:1. Live Performances, 2. Artist I...","[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1. Performances: Live video recordings of arti...
4,Compose a blog post on sustainability practice...,1. Blog post: Outline major sustainability cha...,"1. Blog post, 2. Infographics, 3. Podcast epis...",[INST]Below is an instruction that describes a...,"### Response:1. Blog post, 2. Infographics, 3....","[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","1. Blog post: Writing, 2. Statistics: Research..."


In [ ]:
df_test_with_detail_layer_pred.head(5)

,instruction,output-detail,output-abstract,prompt,response,input_ids,attention_mask,detail-layer-pred
0,Create a virtual museum exhibit on ancient civ...,1. 3D Models: Digitally reconstruct a selectio...,"1. 3D Models, 2. Descriptions, 3. Video Tour.",[INST]Below is an instruction that describes a...,### Response:1. 3D Models: Digitally reconstru...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1. 3D Models of Artifacts:\nCreate high-qualit...
1,Create a multimedia guide to making French pas...,1. Video tutorials: Film detailed videos showi...,"1. Video tutorials, 2. Written recipes, 3. Ste...",[INST]Below is an instruction that describes a...,### Response:1. Video tutorials: Film detailed...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1. Video Tutorials: Record step-by-step video ...
2,Create a social media campaign for a local lib...,1. Posters: Design eye-catching posters with e...,"1. Posters, 2. Video, 3. Reading list, 4. Podc...",[INST]Below is an instruction that describes a...,### Response:1. Posters: Design eye-catching p...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1. Event posters: Design visually appealing po...
3,Compile a digital photo album of a family vaca...,1. Photo Selection: Choose the most memorable ...,"1. Photo Selection, 2. Captions, 3. Background...",[INST]Below is an instruction that describes a...,### Response:1. Photo Selection: Choose the mo...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1. Photo Selection: Choose the best photos fro...
4,Develop a social networking platform for artis...,1. Portfolio tools: Design user-friendly inter...,"1. Portfolio tools, 2. Event announcements, 3....",[INST]Below is an instruction that describes a...,### Response:1. Portfolio tools: Design user-f...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1. Portfolio Creation Tools: Provide artists w...


In [ ]:
import json
import numpy as np


class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [ ]:
# Convert DataFrame to a dictionary for prettier JSON
data_abstr_dict = df_test_with_abstr_layer_pred.to_dict(orient='records')
data_detail_dict = df_test_with_detail_layer_pred.to_dict(orient='records')


dumped_abstr = json.dumps(data_abstr_dict, cls=NumpyEncoder)
dumped_detail = json.dumps(data_detail_dict, cls=NumpyEncoder)


# Writing prettified JSON to a file
with open('data_test_with_abstract_layer_predictions.json', 'w') as f_abstr:
    json.dump(dumped_abstr, f_abstr, indent=4)

with open('data_test_with_detail_layer_predictions.json', 'w') as f_detail:
    json.dump(dumped_detail, f_detail, indent=4)

In [ ]:
from google.colab import files

In [ ]:
files.download('data_test_with_abstract_layer_predictions.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('data_test_with_detail_layer_predictions.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>